# 

This is based on https://learn.deeplearning.ai/langchain/lesson/4/chains lecture

In [1]:
from langchain.document_loaders.github import GitHubIssuesLoader
import os

from dotenv import load_dotenv

load_dotenv()

GHA_TOKEN = os.getenv('GHA')

In [35]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory.buffer import ConversationBufferMemory
chat_llm = ChatOpenAI(temperature=0.8)
memory =ConversationBufferMemory()

LLM chain is one of the simplest and just connects a LLM and Prompt

In [9]:
data = [

{"product":"Apple Mac", "review": "really excellent and fast"},
{"product":"Windward isles bananas", "review": "fresh and tasty"},
{"product":"Cat scarer", "review": "Didn't work, the cats paid it no attention"},
{"product":"Dog collar", "review": "Our dog liked it but slipped his collar and escaped"},
]

In [42]:
getCompanyNameT = PromptTemplate.from_template("For a product {product} think of a good company name in a {style} style")
chain = LLMChain(llm=chat_llm, prompt=getCompanyNameT,verbose=True)

In [43]:
chain.run(product=data[1]['product'], style="jokey")



> Entering new LLMChain chain...
Prompt after formatting:
For a product Windward isles bananas think of a good company name in a jokey style

> Finished chain.


'Banana-nanza Inc.'

The next is a simple sequential chain where we can put the output of 1 message into the input of the next

In [31]:
from langchain.chains import SimpleSequentialChain
## only works with single input and output
getCompanyNameT = PromptTemplate.from_template("For a product {product} think of a good company name in comedy style")
getCompanyTaglineT = PromptTemplate.from_template("For a companyName {companyName} think of a good tagline")
## we don't need to worry about input/output names as there is only 1
chain1 = LLMChain(llm=chat_llm, prompt=getCompanyNameT,verbose=True)
chain2 = LLMChain(llm=chat_llm, prompt=getCompanyTaglineT,verbose=True)

sequential_chain = SimpleSequentialChain(chains=[chain1, chain2])
resp=sequential_chain.run(data[0]['product'])




> Entering new LLMChain chain...
Prompt after formatting:
For a product Apple Mac think of a good company name in comedy style

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
For a companyName iLaughMac think of a good tagline

> Finished chain.


Question - how to get intermediate outputs?

In [45]:
resp

'"Where humor meets innovation on your Mac!"'

Now let's use a regular Sequential Chain to use multiple inputs and outputs.
We need to define the output keys of each subchain

For the sequential chain wrapper we need to define the first 2 inputs and final output.

In [56]:
from langchain.chains import SequentialChain
getCompanyNameT = PromptTemplate.from_template("For a product {product} think of a good company name in {style} style")
getCompanyTaglineT = PromptTemplate.from_template("For a companyName {companyName} think of a good tagline")
sentimentT = PromptTemplate.from_template("for a tagline {tagline} return the sentiment which must be one of Positive, Neutral or Negative")
summaryT = PromptTemplate.from_template("Summarise whether {companyName} with tagline {tagline} and sentiment {sentiment} is a good name")

llm_chain1 = LLMChain(llm=chat_llm, prompt=getCompanyNameT,verbose=True,output_key="companyName")
llm_chain2 = LLMChain(llm=chat_llm, prompt=getCompanyTaglineT,verbose=True, output_key="tagline")
llm_chain3 = LLMChain(llm=chat_llm, prompt=sentimentT,verbose=True, output_key="sentiment")
llm_chain4 = LLMChain(llm=chat_llm, prompt=summaryT,verbose=True, output_key="summary")
sub_chains = [llm_chain1,llm_chain2,llm_chain3,llm_chain4]
seq_chain = SequentialChain(chains=sub_chains, input_variables=["product", "style"],output_variables=["summary"] )
seq_chain.run(product=data[3]['product'], style="silly")




> Entering new LLMChain chain...
Prompt after formatting:
For a product Dog collar think of a good company name in silly style

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
For a companyName Pawsome Puppers think of a good tagline

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
for a tagline "Unleash the Joy of Pawfect Companionship!" return the sentiment which must be one of Positive, Neutral or Negative

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Summarise whether Pawsome Puppers with tagline"Unleash the Joy of Pawfect Companionship!" and sentiment Positive is a good name

> Finished chain.


'The name "Pawsome Puppers" with the tagline "Unleash the Joy of Pawfect Companionship!" has a positive sentiment and conveys a sense of fun and happiness. It suggests a focus on providing excellent companionship and enjoyable experiences with dogs. Overall, it can be considered a good name choice, as it effectively communicates the brand\'s values and appeals to dog lovers.'